In [1]:
import numpy as np
import torch
import torch.nn as nn
import os
import model as m

In [2]:
mo = m.load_torch_hub("resnet18")

Using cache found in C:\Users\netis/.cache\torch\hub\pytorch_vision_v0.10.0


TypeError: __init__() got an unexpected keyword argument 'weights'

In [3]:
torch.hub.load("pytorch/vision:v0.10.0", "resnet18")

Using cache found in C:\Users\netis/.cache\torch\hub\pytorch_vision_v0.10.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
loss = nn.BCELoss()
answer = torch.tensor([1.,1.,0.])
pred = torch.tensor([0.4,0.5,3])
loss(pred, answer)

RuntimeError: all elements of input should be between 0 and 1

In [11]:
import cv2
import numpy as np
img = cv2.imread("datasets/station_true_361_6_4.png")
np.transpose(img, (2,0,1)).shape, img.shape

((3, 384, 384), (384, 384, 3))

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import os
import cv2
import re
import data as d
import model as m
from sklearn.model_selection import KFold, GridSearchCV
from skorch import NeuralNet
import copy

In [2]:
model = m.load_torch_hub("resnet101")
model = m.fit_model_structure(model, 6, 1)

Using cache found in C:\Users\netis/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\netis\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [14]:
model

ResNet(
  (conv1): Sequential(
    (0): Conv2d(6, 64, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (1): Conv2d(64, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=Tr

In [15]:
state_dict = torch.load("best_model/resnet18_best.pt")

In [16]:
for key, val in state_dict.items():
    print(key, val.shape)
    

conv1.0.weight torch.Size([64, 12, 1, 1])
conv1.1.weight torch.Size([64, 64, 7, 7])
bn1.weight torch.Size([64])
bn1.bias torch.Size([64])
bn1.running_mean torch.Size([64])
bn1.running_var torch.Size([64])
bn1.num_batches_tracked torch.Size([])
layer1.0.conv1.weight torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight torch.Size([64])
layer1.0.bn1.bias torch.Size([64])
layer1.0.bn1.running_mean torch.Size([64])
layer1.0.bn1.running_var torch.Size([64])
layer1.0.bn1.num_batches_tracked torch.Size([])
layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight torch.Size([64])
layer1.0.bn2.bias torch.Size([64])
layer1.0.bn2.running_mean torch.Size([64])
layer1.0.bn2.running_var torch.Size([64])
layer1.0.bn2.num_batches_tracked torch.Size([])
layer1.1.conv1.weight torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight torch.Size([64])
layer1.1.bn1.bias torch.Size([64])
layer1.1.bn1.running_mean torch.Size([64])
layer1.1.bn1.running_var torch.Size([64])
layer1.1.bn1.num_batches_tracked torch.Siz

In [87]:
X,y = d.dir_to_img_tensor("datasets")
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5]*6, [0.5]*6), transforms.Resize((448, 448))])
dataset = d.MapDataset(X,y,transform)

In [88]:
kf = KFold(n_splits=3, shuffle=True, random_state = 42)
for train_idx, val_idx in kf.split(dataset):
    print(train_idx, val_idx)

[2 3 4 5] [0 1]
[0 1 3 4] [2 5]
[0 1 2 5] [3 4]


In [107]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [112]:
model_copy2 = model.clone()

AttributeError: 'ResNet' object has no attribute 'clone'

In [103]:
model_copy = copy.deepcopy(model)
model_copy.conv1[0].reset_parameters()

In [61]:
net = NeuralNet(model, max_epochs=20, lr=0.01, criterion=torch.nn.BCELoss, optimizer=torch.optim.Adam,verbose=0,device='cpu')
net.fit(dataset.X, dataset.y)

RuntimeError: Given groups=1, weight of size [64, 6, 1, 1], expected input[4, 384, 384, 6] to have 6 channels, but got 384 channels instead

In [1]:
import torch
import torch.nn as nn

In [5]:
conv = nn.Conv2d(50,100, (7,7),(2,2),(3,3))
data = torch.randn(100,50,448,448)

In [6]:
conv(data).shape

torch.Size([100, 100, 224, 224])

In [22]:
import data
import util
import model as m
recall_dataset = data.load_dataset("small_dataset.pt")
recall_dataset

In [23]:
train_dataset, vd, test_dataset = util.split_data(recall_dataset, 0.9, 0., 0.1)

In [26]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [27]:
kf.split(recall_dataset)

<generator object _BaseKFold.split at 0x0000015B59156430>

In [28]:
from torch.utils.data import Subset, Dataset, DataLoader
train_dataset = recall_dataset
for train_idx, valid_idx in kf.split(recall_dataset):
    print(train_idx.shape, valid_idx.shape)
    print(train_idx, valid_idx)
    cross_train_dataset = Subset(train_dataset, train_idx)
    cross_valid_dataset = Subset(train_dataset, valid_idx)

(350,) (88,)
[  1   2   3   4   5   6   7   8  10  11  12  13  14  16  18  20  21  23
  26  27  28  29  31  32  34  35  36  37  38  40  41  43  44  45  47  48
  49  50  51  52  53  54  58  59  60  61  62  63  64  65  67  68  69  71
  74  80  81  83  84  85  86  87  88  89  91  92  94  95  96  97  98  99
 100 102 103 105 106 107 108 109 110 111 112 114 115 117 118 119 120 121
 122 123 125 127 128 129 130 133 134 135 136 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 156 158 159 160 161 162 163 164 166 167
 168 169 170 171 172 173 174 176 177 178 179 180 182 183 184 185 186 187
 188 189 190 191 193 194 195 196 197 198 199 200 201 202 204 205 206 207
 208 209 210 211 212 213 214 215 216 217 219 221 222 223 224 226 227 228
 229 230 231 232 233 235 236 237 238 239 240 241 242 243 244 246 247 248
 249 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267
 268 269 270 271 272 273 274 275 276 277 279 281 282 283 286 287 288 292
 293 294 295 296 297 298 299 300 301 3